<a href="https://colab.research.google.com/github/ilikemichael/ml1216/blob/main/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import pandas as pd
# pd.options.display.max_rows = 1000
# pd.options.display.max_colwidth =1000

In [2]:
import zipfile
f = zipfile.ZipFile("/content/drive/MyDrive/class/AI python/周凡剛機器學習/chinese_news_trans.zip")
f.extractall()
f = zipfile.ZipFile("/content/drive/MyDrive/class/AI python/周凡剛機器學習/chinese_news_test.zip")
f.extractall()

In [ ]:
import os
import glob
import pandas as pd
def load_data(base):
    datas = {
        "content":[],
        "category":[]
    }
    for dn in glob.glob("{}/*".format(base)):
        small = os.path.join(dn, "*.txt")
        big = os.path.join(dn, "*.TXT")
        for fn in glob.glob(small) + glob.glob(big):
            with open(fn, encoding="utf-8") as f:
                news = f.read()
                datas["content"].append(news)
                cat = os.path.split(dn)[-1]
                datas["category"].append(cat)
                #print(fn)
    return pd.DataFrame(datas)
train = load_data("chinese_news_trans")
test = load_data("chinese_news_test")

In [4]:
test

,content,category
0,日月光華 -- Unix精華區文章閱讀\n-------------------------...,計算機
1,日月光華 -- Virus精華區文章閱讀\n------------------------...,計算機
2,日月光華 -- Programming精華區文章閱讀\n------------------...,計算機
3,日月光華 -- Programming精華區文章閱讀\n------------------...,計算機
4,日月光華 -- HackerVsSecurity精華區文章閱讀\n-------------...,計算機
...,...,...
96,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
97,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
98,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟
99,日月光華 -- Economics精華區文章閱讀\n--------------------...,經濟


In [5]:
{1,2,4,7,3,6,7,1,2,4} #set 不會列出重複
(255, 128, 100) #tuple簡化的字典,自動把位置當欄位
s = "banana" #想把banana每個字串往後移兩格
result = ""
for c in s :
    n = ord(c) + 2
    n = (n-97) % 26 + 97
    result = result + chr(n)
result

'dcpcpc'

In [6]:
#字典的例子,假設有一個字典, 數a出現幾個
dic = {
    "a":5,
    "b":10,
    "c":20
}
new = {}
for k, v in dic.items():
    new[k.upper()] = v
print(new)
print({k.upper():v for k,v in dic.items()})


{'A': 5, 'B': 10, 'C': 20}
{'A': 5, 'B': 10, 'C': 20}


In [7]:
#enmerate 練習
for k in enumerate([30, 50, 60]):
    print(k)
{k:i for i, k in enumerate([20, 50, 60])}

(0, 30)
(1, 50)
(2, 60)


{20: 0, 50: 1, 60: 2}

In [8]:
#這裡開始jieba然後訓練模型
#先把資料標籤數字化
# Series: unique/value_counts
writers = train["category"].unique()
trans = {n:i for i, n in enumerate(writers)}
trans_r = {i:n for i, n in enumerate(writers)}
# Series: replace
y_train = train["category"].replace(trans)
y_test = test["category"].replace(trans)
y_test

0      0
1      0
2      0
3      0
4      0
      ..
96     9
97     9
98     9
99     9
100    9
Name: category, Length: 101, dtype: int64

In [9]:
#找出jieba字典
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")

In [10]:
#定義function newscut,jieba.cut
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train["content"].apply(newscut)
x_test = test["content"].apply(newscut)
x_test

Building prefix dict from /content/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u501edca284da514cb68b53a20324f4e3.cache
Loading model cost 1.628 seconds.
Prefix dict has been built successfully.


0      日月 光華   --   Unix 精華區 文章 閱讀 \n ---------------...
1      日月 光華   --   Virus 精華區 文章 閱讀 \n --------------...
2      日月 光華   --   Programming 精華區 文章 閱讀 \n --------...
3      日月 光華   --   Programming 精華區 文章 閱讀 \n --------...
4      日月 光華   --   HackerVsSecurity 精華區 文章 閱讀 \n ---...
                             ...                        
96     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
97     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
98     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
99     日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
100    日月 光華   --   Economics 精華區 文章 閱讀 \n ----------...
Name: content, Length: 101, dtype: object

In [11]:
#計算次數
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [12]:
#用統計訓練模型
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [13]:
#預測正確分數
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [14]:
#測試
p = input("寫首新聞:")
p = [newscut(p)]
count = vec.transform(p)
prob = clf.predict_proba(count)[0]
for w, pr in zip(writers, prob):
    print(w, round(pr, 4))
ans = clf.predict(count)[0]
print("最可能是:", writers[ans])

寫首新聞:綜合印度部分媒體報導，占星預言家阿南德過去預言提到黃金、石油、天然氣等價格會大幅上漲，但目前僅有金價大好，石油仍未從俄羅斯與沙烏地阿拉伯的石油之戰中，與各國用油量大減的衝擊下中復原，且天然氣、銀價格也下跌。  另外，阿南德引起關注的第三次大戰預言，表示會在今年上演，大戰主要是由美國、印度與伊朗、巴基斯坦在中東的地緣衝突，引爆全球戰事，不過今年緊張衝突中未有阿南德詳細描述的四國戰事，僅有美國狙殺伊朗重要高官，以及伊朗對美軍發射飛彈等零星戰事。  對此，就有不少言論指出，阿南德所預言內容，與多數預言家話術一樣，多以模糊籠統的指涉，好讓他們在事後面臨預言失準質疑之際，有很大空間得以自圓其說。   事實上，阿南德先前預言一一料中，包含世界將會發生大災難、中印邊境之上，比對時間都相當吻合，與他預言大致相合；接著上月又預言本月恐有「全球性災難」，如今英國發現變種的新冠病毒，也被網友認為是印證阿南德所說，但也有多數人認為不要過度迷信。
計算機 0.0
醫藥 0.0
交通 0.0
體育 0.0
環境 0.0
政治 0.0
教育 0.0
藝術 0.0
軍事 0.0
經濟 1.0
最可能是: 經濟
